Convolutional Neural Networks (CNN) are used for classifying images and Computer Vision.

To implement Neural Networks, we will use Tensorflow library.


## Description of the dataset

**Build a CNN to recognize whether the input image is cat or dog**

1) Single Prediction - cat and dog image

2) Training - 4000 images of cat and 4000 images of dog

3) Test set - 1000 images of cat and 1000 images of dog

Total 10,000 images

**As the dataset is very big, we will not implement all on Google Collab**

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
tf.__version__

'2.5.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training Set

In [4]:
## Apply transformations on the images of the Training Set - to avoid overfitting

# these transformations are geometrical like zoom in/out, rotate, flips - image augmentation
# in this way our CNN wont be over trained on these images 

# Keras is a deep learning library which is integrated into Tensorflow 

In [5]:
# Create train_datagen object 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# rescale - feature scaling applied on the pixels (absolutetly necessary for neural networks)

# Import the training dataset 
training_set = train_datagen.flow_from_directory('image_dataset/training_set',  # change path of train set
                                                 target_size = (64, 64), # final size of image to be fed into CNN
                                                 batch_size = 32, # how many images in a batch to train at a time 
                                                 class_mode = 'binary') # binary or categorical (multiclass) classification

Found 7999 images belonging to 2 classes.


### Preprocessing the Training Set

In [6]:
# New images - when deploying our model in production

# we have to the images of the test set intact --> so NO TRANSFORMATION!

test_datagen = ImageDataGenerator(rescale = 1./255) # test set must also be feature scaled 

test_set = test_datagen.flow_from_directory('image_dataset/test_set', # change path of test set 
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
# Sequential library is a part of Keras library. Keras library is integrated into Tenserflow

cnn = tf.keras.models.Sequential() # intialize the network, cnn object

### Step 1 - Convolution

In [8]:
# Create the fully connected layer object. 
# using Convolutional class from Keras (Tensorflow) - tf.keras.Conv2D 

# filters --> no. of feature detectors to apply to our image
# kernel_size --> size of the filter (rows and cols) e.g. size = 3 -> means 3x3
# rectifier function - 'relu' (activation function)

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# input_shape --> specifiy shape of inputs images (1st time)
# coloured image --> input_shape=[X, X, 3]
# B/W image --> input_shape=[X, X, 1]

### 1) Feature Map Created

### Step 2 - Pooling (Max Pooling)

In [9]:
# Apply MaxPool from Keras 

# pool_size --> 2 (which means 2x2)
# strides -> shift pooling frame by 2 steps (2x2 pixels)

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### 2) Pooled Feature Map Created

### Adding a second convolutional layer

In [10]:
# Convolutional + Max Pooling for another layer

# remove input_shape parameter (it exists only in 1st layer)

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) 
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# we can add more layers. More layers is better, but it increases computational costs
# also shape of layers must be same

### Step 3 - Flattening

In [11]:
# Create the Flattening layers (1D layers) by Flattern class from Keras

cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
# Now create a full connected layer by Dense class  

# units --> no. of hidden neurons (a larger number is preferred for image classfication)
# activation function --> rectifier function 

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [13]:
# change units and activation function

# Binary classification - 0 or 1, so dimension of neuron is 1 (1 output neuron)

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # sigmoid activation function (gives probability)

# for multiclass classification, output layer dimensions will depend on no.of classes

# for binary classification - activation function is 'sigmoid'
# for multiclass classification - activation function is 'softmax'

## Part 3 - Training the CNN

### Compiling the CNN 

In [14]:
# using optimizer and loss function, with an evaluation metric

# best optimizer - stochastic gradient descent (adam optimizer) [reduces loss error]
# loss function, i.e. Cost function J theta - difference between prediction and real result

# for binary classification - loss function is 'binary_crossentropy'
# for multiclass classification - loss function is 'categorical_crossentropy'

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set 

In [15]:
# Train and Test at the same time

# same fit method, but neural networks will have extra hyperparameters
# no. of epocs --> no. of iterations the neural network is trained (improves accuracy over time) (make it small as we have 10,000 images)

cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 94s 371ms/step - loss: 0.6739 - accuracy: 0.5783 - val_loss: 0.6239 - val_accuracy: 0.6645
Epoch 2/25
250/250 [==============================] - 85s 339ms/step - loss: 0.6172 - accuracy: 0.6596 - val_loss: 0.5722 - val_accuracy: 0.7050
Epoch 3/25
250/250 [==============================] - 78s 312ms/step - loss: 0.5666 - accuracy: 0.7085 - val_loss: 0.5854 - val_accuracy: 0.6930
Epoch 4/25
250/250 [==============================] - 71s 283ms/step - loss: 0.5405 - accuracy: 0.7290 - val_loss: 0.5226 - val_accuracy: 0.7585
Epoch 5/25
250/250 [==============================] - 65s 261ms/step - loss: 0.5245 - accuracy: 0.7393 - val_loss: 0.5270 - val_accuracy: 0.7435
Epoch 6/25
250/250 [==============================] - 68s 270ms/step - loss: 0.4907 - accuracy: 0.7548 - val_loss: 0.5102 - val_accuracy: 0.7615
Epoch 7/25
250/250 [==============================] - 60s 240ms/step - loss: 0.4768 - accuracy: 0.7710 - val_loss: 0.4841 - val_ac

Therefore, 

**Training Accuracy** = 0.8864

**Test Accuracy** = 0.8000

## Part 4 - Making a single prediction


In [16]:
from keras.preprocessing import image 

# load the images - give path of image
# image size must be the same as the one used in training the model
test_image = image.load_img('image_dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) 

# Conver the images from pixels to numpy array
test_image = image.img_to_array(test_image) 

# CNN was not trained on single image, but on batch of images. This must also be specified for predict method
# Place the image in a batch (extra dimension)
test_image = np.expand_dims(test_image, axis = 0) # axis = 0 --> 1st dimension

result = cnn.predict(test_image)

# The way to figure out 0 or 1, is to call the class indices attritbute from our training set object
training_set.class_indices

# result also has batch dimension - result[0][0] - access batch, then single element within the batch
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [17]:
print(prediction)

dog
